In [ ]:
%matplotlib inline
import sys
import pymc as pm
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
from datetime import datetime
from jtwc import load_jtwc_data
plt.rcParams['figure.figsize'] = (10, 8)



In [ ]:
input_path = r"X:\georisk\HaRIA_B_Wind\data\raw\from_jtwc\bsh"
output_path = r"X:\georisk\HaRIA_B_Wind\projects\tcha\data\derived\windradii"

In [ ]:
df = load_jtwc_data(input_path)

Set up the predictors and predictand. We choose the absolute value of the latitude so that conceptually the model is independent of the hemisphere. The model is defined as $ln(R_{mw}) = \alpha + \beta_0 \Delta p + \beta_1 |\lambda| + \epsilon$, where $\epsilon$ is an error term.

In [ ]:
mask = ~np.isnan(df.r34.values)
X = np.column_stack((df.dP.values[mask], np.abs(df.Latitude.values[mask])))
y = np.log(df.rMax.values[mask])

We set up the model with fairly uninformative priors - all parameters are initialised with a normal distribution with zero mean. The intercept ($\alpha$) is given a broader distribution ($\sigma=10$), while the coefficients for $\Delta p$ and $\lambda$ are chosen to have unit variance. 

In the code below, we specify a prior for the magnitude of the variance of the error term $\epsilon$ - this means we can estimate the variance $\sigma^2$ to use in a $\mathcal{N}(0,\,\sigma^{2})$ distribution.

In [ ]:
Xv = np.column_stack((df.Windspeed.values[mask], np.abs(df.Latitude.values[mask])))

In [ ]:
with pm.Model() as rmaxmodel:
    alpha = pm.Normal(r"$\alpha$", mu=0, sigma=10)
    beta = pm.Normal(r'$\beta$', mu=0, sigma=1, shape=2)
    mu = alpha + beta[0] * X[:, 0] + beta[1] * X[:, 1]
    epsilon = pm.HalfNormal(r"$\epsilon$", sigma=1)
    y_hat = pm.Normal('y_hat', mu=mu, sigma=epsilon, observed=y)
    step = pm.Metropolis()
    trace = pm.sample(10000, tune=5000, step=step, return_inferencedata=True)
    trace.extend(pm.sample_posterior_predictive(trace))

In [ ]:
axes = az.plot_trace(trace, combined=False)
aq = np.quantile(trace.posterior[r'$\alpha$'], [0.05, 0.5, 0.95])
bq = np.quantile(trace.posterior[r'$\beta$'], [0.05, 0.5, 0.95], axis=(0, 1))
eq = np.quantile(trace.posterior[r"$\epsilon$"], [0.05, 0.5, 0.95])

axes[0, 0].axvline(aq[1], ls='--', color='k', label=rf'$\alpha = {{{aq[1]:.2f}}}$')
axes[0, 0].axvline(aq[0], ls='--', color='gray', label=f"90% CI [{aq[0]:.2f}, {aq[2]:.2f}]")
axes[0, 0].axvline(aq[2], ls='--', color='gray')

axes[1, 0].axvline(bq[1, 0], ls='--', color='b', label=rf'$\beta_0 = {{{bq[1, 0]:.4f}}}$ [{bq[0, 0]:.4f}, {bq[2, 0]:.4f}]')
axes[1, 0].axvline(bq[0, 0], ls='--', color='gray',)
axes[1, 0].axvline(bq[2, 0], ls='--', color='gray')

axes[1, 0].axvline(bq[1, 1], ls='--', color='y', label=rf'$\beta_1 = {{{bq[1, 1]:.4f}}}$ [{bq[0, 1]:.4f}, {bq[2, 1]:.4f}]')
axes[1, 0].axvline(bq[0, 1], ls='--', color='gray',)
axes[1, 0].axvline(bq[2, 1], ls='--', color='gray')

axes[2, 0].axvline(eq[1], ls='--', color='k', label=rf'$\epsilon = {{{eq[1]:.3f}}}$')
axes[2, 0].axvline(eq[0], ls='--', color='gray', label=f"90% CI [{eq[0]:.3f}, {eq[2]:.3f}]")
axes[2, 0].axvline(eq[2], ls='--', color='gray')

axes[0, 0].legend(fontsize='x-small')
axes[1, 0].legend(fontsize='x-small')
axes[2, 0].legend(fontsize='x-small')
plt.tight_layout()

In [ ]:
az.summary(trace, round_to=4, hdi_prob=0.9,)

In [ ]:
#trace.posterior['ymodel'] = trace.posterior['alpha'] + trace.posterior['beta'] * xr.DataArray(X) + trace.posterior['error']
plt.scatter(X[:,0], np.exp(trace.posterior_predictive['y_hat'][0, 0, :]), marker='.', alpha=0.25)
plt.scatter(X[:,0], np.exp(trace.posterior_predictive['y_hat'][1, 0, :]), marker='x', alpha=0.25)
plt.scatter(X[:,0], np.exp(trace.posterior_predictive['y_hat'][2, 0, :]), marker='p', alpha=0.25)
plt.scatter(X[:,0], np.exp(trace.posterior_predictive['y_hat'][3, 0, :]), marker='d', alpha=0.25)
plt.scatter(X[:, 0], np.exp(y), marker='x', c='k')
plt.xlabel("Pressure deficit [hPa]")
plt.ylabel(r"$R_{mw}$ [km]")

In [ ]:
plt.scatter(X[:, 1], np.exp(trace.posterior_predictive['y_hat'][0, 0, :]), marker='.', alpha=0.25)
plt.scatter(X[:, 1], np.exp(trace.posterior_predictive['y_hat'][1, 0, :]), marker='x', alpha=0.25)
plt.scatter(X[:, 1], np.exp(trace.posterior_predictive['y_hat'][2, 0, :]), marker='p', alpha=0.25)
plt.scatter(X[:, 1], np.exp(trace.posterior_predictive['y_hat'][3, 0, :]), marker='d', alpha=0.25)
plt.scatter(X[:, 1], np.exp(y), marker='x', c='k')
plt.xlabel(r"Latitude [$^{\circ}$S]")
plt.ylabel(r"$R_{mw}$ [km]")

Now use maximum sustained wind speed as the intensity predictor

In [ ]:
with pm.Model() as rmaxmodel:
    alpha = pm.Normal(r"$\alpha$", mu=0, sigma=10)
    beta = pm.Normal(r"$\beta$", mu=0, sigma=1, shape=2)
    mu = alpha + beta[0] * Xv[:, 0] + beta[1] * Xv[:, 1]
    epsilon = pm.HalfNormal(r"$\epsilon$", sigma=1)
    y_hat = pm.Normal('y_hat', mu=mu, sigma=epsilon, observed=y)
    step = pm.Metropolis()
    trv = pm.sample(10000, tune=5000, step=step, return_inferencedata=True)
    trv.extend(pm.sample_posterior_predictive(trv))

In [ ]:
axes = az.plot_trace(trv, combined=False)
aq = np.quantile(trv.posterior[r"$\alpha$"], [0.05, 0.5, 0.95])
bq = np.quantile(trv.posterior[r"$\beta$"], [0.05, 0.5, 0.95], axis=(0, 1))
eq = np.quantile(trv.posterior[r"$\epsilon$"], [0.05, 0.5, 0.95])

axes[0, 0].axvline(aq[1], ls='--', color='k', label=rf'$\alpha = {{{aq[1]:.2f}}}$')
axes[0, 0].axvline(aq[0], ls='--', color='gray', label=f"90% CI [{aq[0]:.2f}, {aq[2]:.2f}]")
axes[0, 0].axvline(aq[2], ls='--', color='gray')

axes[1, 0].axvline(bq[1, 0], ls='--', color='b', label=rf'$\beta_0 = {{{bq[1, 0]:.4f}}}$ [{bq[0, 0]:.4f}, {bq[2, 0]:.4f}]')
axes[1, 0].axvline(bq[0, 0], ls='--', color='gray',)
axes[1, 0].axvline(bq[2, 0], ls='--', color='gray')

axes[1, 0].axvline(bq[1, 1], ls='--', color='y', label=rf'$\beta_1 = {{{bq[1, 1]:.4f}}}$ [{bq[0, 1]:.4f}, {bq[2, 1]:.4f}]')
axes[1, 0].axvline(bq[0, 1], ls='--', color='gray',)
axes[1, 0].axvline(bq[2, 1], ls='--', color='gray')

axes[2, 0].axvline(eq[1], ls='--', color='k', label=rf'$\epsilon = {{{eq[1]:.3f}}}$')
axes[2, 0].axvline(eq[0], ls='--', color='gray', label=f"90% CI [{eq[0]:.3f}, {eq[2]:.3f}]")
axes[2, 0].axvline(eq[2], ls='--', color='gray')

axes[0, 0].legend(fontsize='x-small')
axes[1, 0].legend(fontsize='x-small')
axes[2, 0].legend(fontsize='x-small')
plt.tight_layout()

In [ ]:
az.summary(trv, round_to=4, hdi_prob=0.9,)

In [ ]:
plt.scatter(Xv[:,0], np.exp(trv.posterior_predictive['y_hat'][0, 0, :]), marker='.', alpha=0.25)
plt.scatter(Xv[:,0], np.exp(trv.posterior_predictive['y_hat'][1, 0, :]), marker='x', alpha=0.25)
plt.scatter(Xv[:,0], np.exp(trv.posterior_predictive['y_hat'][2, 0, :]), marker='p', alpha=0.25)
plt.scatter(Xv[:,0], np.exp(trv.posterior_predictive['y_hat'][3, 0, :]), marker='d', alpha=0.25)
plt.scatter(Xv[:, 0], np.exp(y), marker='x', c='k')
plt.xlabel("Wind speed [kts]")
plt.ylabel(r"$R_{mw}$ [km]")

In [ ]:
plt.scatter(Xv[:, 1], np.exp(trv.posterior_predictive['y_hat'][0, 0, :]), marker='.', alpha=0.25)
plt.scatter(Xv[:, 1], np.exp(trv.posterior_predictive['y_hat'][1, 0, :]), marker='x', alpha=0.25)
plt.scatter(Xv[:, 1], np.exp(trv.posterior_predictive['y_hat'][2, 0, :]), marker='p', alpha=0.25)
plt.scatter(Xv[:, 1], np.exp(trv.posterior_predictive['y_hat'][3, 0, :]), marker='d', alpha=0.25)
plt.scatter(Xv[:, 1], np.exp(y), marker='x', c='k')
plt.xlabel(r"Latitude [$^{\circ}$S]")
plt.ylabel(r"$R_{mw}$ [km]")

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1, 2, sharey=True, figsize=(12, 8))
sns.kdeplot(x=Xv[:, 1], y = np.exp(trv.posterior_predictive['y_hat'][0, 0, :]),
            fill=True, ax=ax[0], cbar=True, cmap='viridis',
            cbar_kws={'orientation': 'horizontal', 'format': "{x:.2e}"})
sns.kdeplot(x=Xv[:, 1], y = np.exp(y), ax=ax[0], color='k')
c_bar = ax[0].collections[0].colorbar
c_bar.ax.tick_params(rotation=90)
sns.kdeplot(x=Xv[:, 0], y = np.exp(trv.posterior_predictive['y_hat'][0, 0, :]),
            fill=True, ax=ax[1],  cbar=True, cmap='viridis',
            cbar_kws={'orientation': 'horizontal', 'format': "{x:.2e}"})
sns.kdeplot(x=Xv[:, 0], y = np.exp(y), ax=ax[1], color='k')
c_bar = ax[1].collections[0].colorbar
c_bar.ax.tick_params(rotation=90)
ax[0].grid()
ax[1].grid()
ax[0].set_ylim((0, 150))
ax[0].set_ylabel(r"$R_{mw}$ [km]")
ax[0].set_xlabel(r"Latitude [$^{\circ}$S]")
ax[1].set_xlabel("Wind speed [kts]")
fig.tight_layout()